In [1]:
import nltk
import string
import csv
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from gensim import corpora, models
import pandas as pd

In [2]:
df  = pd.read_csv("Homework_1.csv",delimiter="^",header=None)
df2 = df[df[0]==8].sort_values(3)[[0,3,5]]
df3 = pd.DataFrame(df2.groupby(3,as_index=True).apply(lambda x: x.sum())[5]).reset_index()
df3

,3,5
0,2015-10-15,essential for good living!! I love it!!
1,2015-10-23,This was my first attempt at grinding. I was ...
2,2015-10-25,It works well but the container is too small ...
3,2015-11-11,We own two of these. One for spices and one f...
4,2015-11-16,this is a Christmas present to someone...so d...
5,2015-11-19,Does the job perfectly. Nice shape for pourin...
6,2015-11-21,"Since I bought it, I have used it almost ever..."
7,2015-11-22,Perfect!
8,2015-11-23,I use it every morning. Works great! I put 8 ...
9,2015-12-01,Perfect Grinder! Now thats coffee This makes ...


In [3]:
token_dict = {}
for date , comment in zip(df3[3],df3[5]):
    lowers = comment.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[date] = no_punctuation
decode_token = [w.decode("utf-8") for w in token_dict.values()]

In [4]:
doc_tokens = [nltk.word_tokenize(doc) for doc in decode_token]
ps = PorterStemmer()
doc_stemmed = []
for one_doc in doc_tokens:
    doc_filtered = [w for w in one_doc if not w in stopwords.words("english")]
    doc_stem = [ps.stem(w).encode("utf-8") for w in doc_filtered]
    doc_stemmed.append(doc_stem)

In [5]:
dictionary = corpora.Dictionary(doc_stemmed)
corpus = [dictionary.doc2bow(text) for text in doc_stemmed]
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes = 20,chunksize=10000)

In [6]:
ldamodel.print_topics(num_topics=10, num_words=2)

[(0, u'0.017*"well" + 0.017*"work"'),
 (1, u'0.040*"coffe" + 0.038*"grind"'),
 (2, u'0.031*"time" + 0.027*"unit"'),
 (3, u'0.018*"wife" + 0.018*"love"'),
 (4, u'0.027*"use" + 0.014*"coffe"'),
 (5, u'0.033*"coffe" + 0.026*"grind"'),
 (6, u'0.030*"great" + 0.027*"small"'),
 (7, u'0.027*"one" + 0.014*"use"'),
 (8, u'0.049*"grind" + 0.041*"coffe"'),
 (9, u'0.036*"grinder" + 0.028*"great"')]

In [7]:
raw_topics = [s for n,s in ldamodel.print_topics(num_topics=10, num_words=2)]
strip = '0123456789*."'
topics = [''.join([i for i in s.encode().replace(" +",",") if not i in strip]).replace(" +", ",") for s in raw_topics]

In [8]:
topics

['well, work',
 'coffe, grind',
 'time, unit',
 'wife, love',
 'use, coffe',
 'coffe, grind',
 'great, small',
 'one, use',
 'grind, coffe',
 'grinder, great']

In [9]:
prob_1 = []
prob_2 = []
for i in range(len(corpus)):
    df4 = pd.DataFrame(zip(topics,[p for t, p in ldamodel.get_document_topics(corpus[i],minimum_probability=1e-4)]),
                      columns = ["topics","prob"])
    df4.sort_values(by = "prob", ascending=0,inplace=True)
    df4 = df4.head(2)
    df4["top_two"] = df4.prob.map(str) + "*[" + df4.topics + "]"
    prob_1.append(df4.top_two.values[0])
    prob_2.append(df4.top_two.values[1])
result = pd.DataFrame(zip(df3[3],prob_1,prob_2)
                      ,columns=(["Time"," Probability*Topic 1", "Probability*Topic 2"]))
result

,Time,Probability*Topic 1,Probability*Topic 2
0,2015-10-15,"0.819981784909*[one, use]","0.0200046660537*[grinder, great]"
1,2015-10-23,"0.935707885097*[coffe, grind]","0.00714583795912*[time, unit]"
2,2015-10-25,"0.943740592339*[one, use]","0.00625389464639*[grind, coffe]"
3,2015-11-11,"0.992372504731*[grind, coffe]","0.000847575330308*[coffe, grind]"
4,2015-11-16,"0.629856899509*[grind, coffe]","0.366040217268*[great, small]"
5,2015-11-19,"0.871423553588*[grind, coffe]","0.0142877374403*[coffe, grind]"
6,2015-11-21,"0.699983556188*[great, small]","0.0333392556278*[grinder, great]"
7,2015-11-22,"0.978046444617*[grinder, great]","0.00243993332649*[grind, coffe]"
8,2015-11-23,"0.924989061304*[coffe, grind]","0.00833704090848*[coffe, grind]"
9,2015-12-01,"0.989772174597*[coffe, grind]","0.00113649122165*[great, small]"
